In [6]:
!pip install -U langchain-community

In [7]:
!pip install chromadb

In [8]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('cointegrated/rubert-tiny2')
sentences = ["привет мир", "hello world", "здравствуй вселенная"]
embeddings = model.encode(sentences)
print(len(embeddings[0]))


312


In [10]:
#for folder

#loder = DirectoryLoader("/content/TheStarryNight.txt", glob='*/.txt',loader_cls=TextLoader)

In [11]:
loader = TextLoader('/content/TheStarryNight.txt')
document = loader.load()

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(document)

In [13]:
texts[0]

Document(metadata={'source': '/content/TheStarryNight.txt'}, page_content="The Starry Night is an oil-on-canvas painting by the Dutch Post-Impressionist painter Vincent van Gogh, painted in June 1889. It depicts the view from the east-facing window of his asylum room at Saint-Rémy-de-Provence, just before sunrise, with the addition of an imaginary village.[1][2][3] It has been in the permanent collection of the Museum of Modern Art in New York City since 1941,[4] acquired through the Lillie P. Bliss Bequest. Widely regarded as Van Gogh's magnum opus,[5] The Starry Night is one of the most recognizable paintings in Western art.[6][7]\n\nThe asylum")

In [14]:
len(texts)

36

#creating Chromadb


In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings

persist_directory = 'db'

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

<ipython-input-16-9832c5518aee>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(


In [17]:
vectordb = Chroma.from_documents(texts, hf, persist_directory=persist_directory)

In [18]:
vectordb.persist()
vectordb=None

<ipython-input-18-fb7efa6acf19>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [19]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=hf)

<ipython-input-19-42a0321c1026>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=hf)


In [20]:
retriever = vectordb.as_retriever()

In [21]:
retriever.get_relevant_documents("What is the meaning of life?")

<ipython-input-21-9ad0b72aad76>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is the meaning of life?")


[Document(metadata={'source': '/content/TheStarryNight.txt'}, page_content='Soon after he arrives in Arles in February 1888, Van Gogh wrote to Theo, "I need a starry night with cypresses or—perhaps above a field of ripe wheat; there are some really beautiful nights here." That same week, he wrote to Bernard, "A starry sky is something I should like to try to do, just as in the daytime I am going to try to paint a green meadow spangled with dandelions."[38] He compared the stars to dots on a map and mused that, as one takes a train to travel on Earth, "we take death to reach a star."[39] Although at this point in his life Van Gogh was disillusioned by religion,[40][41] he appears not to have lost his belief in an afterlife. He voiced this ambivalence in a letter to Theo after having painted Starry Night Over the Rhône, confessing to a "tremendous need for, shall I say the word—for religion—so I go outside at night to paint the stars."[42]'),
 Document(metadata={'source': '/content/TheSt

In [22]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [23]:
retriever.get_relevant_documents("What is the meaning of life?")

[Document(metadata={'source': '/content/TheStarryNight.txt'}, page_content='Soon after he arrives in Arles in February 1888, Van Gogh wrote to Theo, "I need a starry night with cypresses or—perhaps above a field of ripe wheat; there are some really beautiful nights here." That same week, he wrote to Bernard, "A starry sky is something I should like to try to do, just as in the daytime I am going to try to paint a green meadow spangled with dandelions."[38] He compared the stars to dots on a map and mused that, as one takes a train to travel on Earth, "we take death to reach a star."[39] Although at this point in his life Van Gogh was disillusioned by religion,[40][41] he appears not to have lost his belief in an afterlife. He voiced this ambivalence in a letter to Theo after having painted Starry Night Over the Rhône, confessing to a "tremendous need for, shall I say the word—for religion—so I go outside at night to paint the stars."[42]'),
 Document(metadata={'source': '/content/TheSt

#Chain

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [25]:
!pip install langchain-huggingface


In [26]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="microsoft/DialoGPT-small",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    ),
)

chat_model = ChatHuggingFace(llm=llm)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [27]:
chain_qa = RetrievalQA.from_chain_type(llm=chat_model, chain_type="stuff", retriever=retriever,return_source_documents=True)